In [ ]:
# Package imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
data = pd.read_csv("data/data.csv")
print("data type:",type(data))
print("data shape:", data.shape)
print("data:")
print(data.head().to_string(index=False))

# Split the data into train and test
train_dataset = data.sample(frac=0.8, random_state=0)
test_dataset = data.drop(train_dataset.index)

# Normalize (divide positions (x1,x3,x5,0-5) by 5 and values (x2,x4,x6,0-100) by 100)
x_train = train_dataset.multiply([1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100]).iloc[:,0:6]
y_train = train_dataset.multiply([1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100]).iloc[:,6:12]
x_test = test_dataset.multiply([1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100]).iloc[:,0:6]
y_test = test_dataset.multiply([1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100]).iloc[:,6:12]
print("Train data shape:", train_dataset.shape)
print("Test data shape:", test_dataset.shape)
print("Train data x shape:",x_train.shape)
print("Train data y shape:",y_train.shape)

In [ ]:
# Definde Sequential Model
model = keras.Sequential(
    [   keras.Input(shape=(6,)),
        keras.layers.Dense(48, activation='relu'),
        keras.layers.Dense(96, activation='relu'),
        keras.layers.Dense(192, activation='relu'),
        keras.layers.Dense(96, activation='relu'),
        keras.layers.Dense(6)
    ]
)
model.summary()

In [ ]:
# loss, optimizer and metrics
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['mae'])

In [ ]:
class haltCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss') <= 1.0e-06):
            print("\n\n\nReached loss limit, stopping training!\n\n\n")
            self.model.stop_training = True
trainingStopCallback = haltCallback()

In [ ]:
# training
model.fit(x_train, y_train, epochs=100,batch_size=64, verbose=1, callbacks=[trainingStopCallback])

In [ ]:
# Predict value using the first example in x_test.
# If the model is trained well, the prediction should display the same values as the values in y_test.
x1 = x_test.iloc[0:1,:]
y1 = y_test.iloc[0:1,:]
x1_pred = model.predict(x1, verbose=0)
print('x1',(x1*[5,100,5,100,5,100]).astype(int).to_string(index=False))
print('y1',(y1*[5,100,5,100,5,100]).astype(int).to_string(index=False))
print('prediction: ',np.round(x1_pred*[5,100,5,100,5,100]).astype(int))


In [ ]:
pred = model.predict(x_test)
# De-Normalize
x_round = np.round(x_test*[5,100,5,100,5,100]).astype(int)
y_round = np.round(y_test*[5,100,5,100,5,100]).astype(int)
pred_round = np.round(pred*[5,100,5,100,5,100]).astype(int)
test_result = pred_round - y_round
test_result_row_sums = test_result.abs().sum(axis=1)
num_correct= len(test_result_row_sums[test_result_row_sums == 0])
print('Number of correct solutions:',num_correct,', out of ',test_result.shape[0], (100*num_correct)/test_result.shape[0], '%')

In [ ]:
model = keras.models.load_model("mathpyramid_1987_correct")
model.summary()

In [ ]:
# Predict with higher values (<= 1000) -> works too
data = {'x1': [0/5],
        'x2': [175/1000],
        'x3': [1/5],
        'x4': [114/1000],
        'x5': [2/5],
        'x6': [102/1000]
        }  
# Create the pandas DataFrame
df = pd.DataFrame(data)
print(df)
pred = model.predict(df)
print(np.round(pred*[5,1000,5,1000,5,1000]).astype(int))